In [177]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [180]:
#Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [149]:
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [150]:
#Encode categorical variables
one_hot_encoder_gen = OneHotEncoder(sparse=False)
gen_encoder=one_hot_encoder_gen.fit_transform(data[['Gender']])
gen_encoder 

one_hot_encoder_geo = OneHotEncoder(sparse=False)
geo_encoder=one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [151]:
##these are the column names
one_hot_encoder_geo.get_feature_names_out(['Geography'])
one_hot_encoder_gen.get_feature_names_out(['Gender'])

array(['Gender_Female', 'Gender_Male'], dtype=object)

In [152]:
gen_encoded_df = pd.DataFrame(gen_encoder, columns=one_hot_encoder_gen.get_feature_names_out(['Gender']))
gen_encoded_df
geo_encoded_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [153]:
#drop the Grography and Gender column
data = data.drop(['Geography','Gender'], axis=1)
data


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0
9996,516,35,10,57369.61,1,1,1,101699.77,0
9997,709,36,7,0.00,1,0,1,42085.58,1
9998,772,42,3,75075.31,2,1,0,92888.52,1


In [154]:
data = pd.concat([data,gen_encoded_df,geo_encoded_df], axis=1)
data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,1.0,0.0,0.0
1,608,41,1,83807.86,1,0,1,112542.58,0,1.0,0.0,0.0,0.0,1.0
2,502,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,1.0,0.0,0.0
3,699,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,1.0,0.0,0.0
4,850,43,2,125510.82,1,1,1,79084.10,0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0.0,1.0,1.0,0.0,0.0
9996,516,35,10,57369.61,1,1,1,101699.77,0,0.0,1.0,1.0,0.0,0.0
9997,709,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,1.0,0.0,0.0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0,1.0,0.0


In [155]:
#Save the encoders

with open('onehot_encoder_gender.pkl', 'wb') as file:
     pickle.dump(one_hot_encoder_gen, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
     pickle.dump(one_hot_encoder_geo, file)


In [158]:
#divide the data into dependent and independent
X = data.drop('Exited', axis=1)
y = data['Exited']

In [160]:
#Split the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# #Scale the features
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.35649971, -0.6557859 ,  0.34567966, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.29493847, -0.3483691 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.3900109 ,  1.03972843, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  1.15059039, -1.38944225, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [157]:
X_train.shape, X_test.shape

((8000, 11), (2000, 11))

In [161]:
#save the scaled file in pickle form
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN implementation

In [162]:
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [168]:
y_test.shape

(2000,)

In [169]:
#Model building
model=Sequential()


#hidden layer1
model.add(Dense(100, activation='sigmoid', input_shape=(13,)))
model.add(Dropout(0.3))
#hidden layer2
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(1, activation='sigmoid'))



In [170]:
import tensorflow
opt =tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy
loss

keras.src.losses.BinaryCrossentropy

In [171]:
#Compile the model - to do forward and backwrd propagation
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [172]:
#Set up the tensorboard to visualise the logs that we have when training the entire model
log_dir = "logs/fit/" +datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [173]:
#Setup Early stopping
early_stopping_callback = EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights=True)

In [174]:
#Train the model - apply the callsback here
history=model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                   callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4718 - accuracy: 0.7922 - val_loss: 0.4163 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4560 - accuracy: 0.7972 - val_loss: 0.4165 - val_accuracy: 0.8075
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4535 - accuracy: 0.7965 - val_loss: 0.4236 - val_accuracy: 0.8155
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4442 - accuracy: 0.8090 - val_loss: 0.4014 - val_accuracy: 0.8305
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4293 - accuracy: 0.8186 - val_loss: 0.3842 - val_accuracy: 0.8390
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4274 - accuracy: 0.8216 - val_loss: 0.3792 - val_accuracy: 0.8425
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4112 - accuracy: 0.8236 - val_loss: 0.3770 - val_accuracy: 0.8340

In [175]:
#load the entire tensorboard
model.save("model.h5")


c:\Users\psharma\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [108]:
#load the entire tensorboard extension
%reload_ext tensorboard

In [176]:
%tensorboard --logdir logs/fit --port=6006

Reusing TensorBoard on port 6006 (pid 15112), started 8:01:40 ago. (Use '!kill 15112' to kill it.)

In [57]:
#! pip list

In [ ]:
##Load the pickle files-there are three pickle files that we have here.Let's do this in prediction.ipynb file.

